# Transfer Learning

Transfer learning: repurposing a network for a different task.

Bryan Catanzaro: wrote library than became CUDNN. Lol.

GPU: Optimisod far throughput computation (simult)
    - since update lots of pixels on screen at the same time.
    - dl computations involve a lot of parallelisation.
CPU: latency -> running a single thread of instructions as quickly as possible

Rule of thumb: GPU 5x faster than CPU.


Stanford University Notes on Transfer Learning: http://cs231n.github.io/transfer-learning/


Why transfer learning is useful
* Can use intel stored in trained networks to accelerate own progress
- Your dataset may be small, so networks pre-trained on much data can help you generalise.

DL more popular recently because of
- Increased availability of labelled data
- Increase in computational power

- 1990s LeNet
- ImageNet image classification competition 
    - 2012 AlexNet (Fundamental architecture
        - Used parallelism of 40-byte GPUs to train network in about a week.
        - Pioneered use of ReLUs in activation.
        - Dropout to avoid overfitting.
        - 15% error vs 26% error of winner in 2011.

## Labs
Transfer Learning with TensorFlow
Transfer learning is the practice of starting with a network that has already been trained, and then applying that network to your own problem.

Because neural networks can often take days or even weeks to train, transfer learning (i.e. starting with a network that somebody else has already spent a lot of time training) can greatly shorten training time.

So how we do go about applying transfer learning? Two popular methods are feature extraction and finetuning.

- Feature extraction. We take a pretrained neural network and replace the final layer (classification layer) with a new classification layer for the new dataset or perhaps even a small neural network (eventually has a classification layer). During training the weights in all the old layers are frozen, only the weights for the new layer we added are trained. In other words, the gradient doesn't flow backwards past the first new layer we add.

- Finetuning. Essentially the same as feature extraction except the weights of the old model aren't frozen. The network is trained end-to-end.
In this lab and the one later in the section we're going to focus on feature extraction since it's less computationally intensive.

### Feature Extraction

The problem is that AlexNet was trained on the ImageNet database, which has 1000 classes of images. You can see the classes in the caffe_classes.py file. None of those classes involves traffic signs.

In order to successfully classify our traffic sign images, you need to remove the final, 1000-neuron classification layer and replace it with a new, 43-neuron classification layer.

This is called feature extraction, because you're basically extracting the images features captured by the penultimate layer, and passing them to a new classification layer.

#### Solution
First, I figure out the shape of the final fully connected layer, in my opinion this is the trickiest part. To do that I have to figure out the size of the output from fc7. Since it's a fully connected layer I know it's shape will be 2D so the second (or last) element of the list will be the size of the output. fc7.get_shape().as_list()[-1] does the trick. I then combine this with the number of classes for the Traffic Sign dataset to get the shape of the final fully connected layer, shape = (fc7.get_shape().as_list()[-1], nb_classes). The rest of the code is just the standard way to define a fully connected in TensorFlow. Finally, I calculate the probabilities via softmax, probs = tf.nn.softmax(logits).

In [ ]:
import time
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.misc import imread
from alexnet import AlexNet

sign_names = pd.read_csv('signnames.csv')
nb_classes = 43

x = tf.placeholder(tf.float32, (None, 32, 32, 3))
resized = tf.image.resize_images(x, (227, 227))

# NOTE: By setting `feature_extract` to `True` we return
# the second to last layer.
fc7 = AlexNet(resized, feature_extract=True)

# Define a new fully connected layer followed by a softmax activation to classify
# the traffic signs. Assign the result of the softmax activation to `probs` below.

# use this shape for the weight matrix
shape = (fc7.get_shape().as_list()[-1], nb_classes)  
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))

# this is not a RELU layer since there is no nonlinear activation.
logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)
probs = tf.nn.softmax(logits)

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# Read Images
im1 = imread("construction.jpg").astype(np.float32)
im1 = im1 - np.mean(im1)

im2 = imread("stop.jpg").astype(np.float32)
im2 = im2 - np.mean(im2)

# Run Inference: of course the results would be unreliable
t = time.time()
output = sess.run(probs, feed_dict={x: [im1, im2]})

# Print Output
for input_im_ind in range(output.shape[0]):
    inds = np.argsort(output)[input_im_ind, :]
    print("Image", input_im_ind)
    for i in range(5):
        print("%s: %.3f" % (sign_names.ix[inds[-1 - i]][1], output[input_im_ind, inds[-1 - i]]))
    print()

print("Time: %.3f seconds" % (time.time() - t))

### Training the Feature Extractor
The feature extractor you just created works, in the sense that data will flow through the network and result in predictions.

But the predictions aren't accurate, because you haven't yet trained the new classification layer.

In order to do that, you'll need to read in the training dataset and train the network with cross entropy.

Training AlexNet (even just the final layer!) can take a little while, so if you don't have a GPU, running on a subset of the data is a good alternative. As a point of reference one epoch over the training set takes roughly 53-55 seconds with a GTX 970.



Simplified AlexNet used as a starting point for CV today

Others:
- Some feature

### VGGNet (VGG)

From Visual Geometry group at Oxford.

Architecture:
* Long series of 3x3 convs 
* broken up by series of 2x2 pooling layers
* finished by a trio of fully connected layers at the end.

Starting point for working on other image classifcation tasks.
VGG Strengths: 
* Flexibility

Claim: Bias towards action needed to be a successful deep learning practitioner. Experiment continually.

### GoogLeNet

* Runs really fast.
* Inception module: trains well and is efficiently deployable.
    - At each layer of your convnet, you can make a choice - pooling or convolution (1x1 or 3x3 or 5x5?) -> use all of them instead of choosing only one.
    - Composition of averaging pooling followed by 1x1, then 1x1 followed by 3x3 ... and then concatenate the output.
    - Choose params in such a way such that tho total number of parameters is small (so GoogLeNet runs nearly as fast as AlexNet,, can run in real time).
    - -> Like ensembles.

### ResNet (Microsoft)
* 152 layers (vs AlexNet 8 layers, VGG 19, GoogLeNet 22)
* Same structure repeated again and again like VGG. -> Add connections to NN that skill layers so
* 3% on ImageNet, > human accuracy.